## Parte 1 ● Ejecutar los scripts con una muestra relativamente pequeña de cada uno de losendpoints y verificar que todo funcione correctamente.

https://www.ree.es/es/datos/apidatos




In [6]:
import requests
import pandas as pd
import numpy as np
from pprint import pprint

In [7]:
# Versiones

print(f"numpy=={np.__version__}")
print(f"pandas=={pd.__version__}")
print(f"requests=={requests.__version__}")

numpy==1.26.4
pandas==2.2.2
requests==2.32.3


In [8]:
# Constantes comunes
URL = "https://apidatos.ree.es" # URL base de la API
HEADERS = {
    "Accept": "application/json",
    "Content-Type": "application/json",
    "Host": "apidatos.ree.es",
    "User-Agent": "Mozilla/5.0"
}

# Diccionario de geo_ids y nombres de regiones
region_ids = {
    "8741": "peninsular",
    "8742": "canarias",
    "8743": "baleares",
    "8744": "ceuta",
    "8745": "melilla",
    "4": "andalucia",
    "5": "aragon",
    "6": "cantabria",
    "7": "castilla la mancha",
    "8": "castilla y leon",
    "9": "cataluña",
    "10": "pais vasco",
    "11": "principado de asturias",
    "13": "comunidad de madrid",
    "14": "comunidad de navarra",
    "15": "comunidad de valenciana",
    "16": "extremadura",
    "17": "galicia",
    "20": "la rioja",
    "21": "region de murcia"
}

BALANCE

In [44]:
### BALANCE-ELECTRICO ###

# Endpoint para el balance eléctrico
filtro_balance = '/es/datos/balance/balance-electrico'

endpoint = f"{URL}/{filtro_balance}"

# Inicializamos un DataFrame vacío para almacenar los datos
df_balance = pd.DataFrame()

# Iteramos sobe los años de 2019 a 2024 y extraemos los datos de balance
# de cada región para cada año
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "day"

    print(f"Extrayendo datos de balance del año {year}...")

    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        data = response.json()
        included = data.get('included', [])

        registros = []
        # Extraemos los datos de balance
        # para cada grupo de energía
        # y cada indicador
        for grupo in included:
            group_name = grupo['attributes']['title']
            contenidos = grupo['attributes'].get('content', [])

            for item in contenidos:
                indicador = item['attributes']['title']
                valores = item['attributes'].get('values', [])
                # Extraemos los valores para cada indicador
                # y los almacenamos en el DataFrame
                for punto in valores:
                    registros.append({
                        'fecha': punto['datetime'],
                        'valor': punto['value'],
                        'tipo': indicador,
                        'energia': group_name,
                        'region': region_name,
                    })
        # Convertimos los registros a un DataFrame
        # y lo concatenamos al DataFrame principal
        df_balance_year = pd.DataFrame(registros)
        df_balance = pd.concat([df_balance, df_balance_year], ignore_index=True)

print('Datod de balance entre 2019 y 2024 extraidos:')
display(df_balance.head())
# Guardamos el DataFrame en un archivo CSV
df_balance.to_csv('../data/sinprocesar/balance/balance-electrico.csv', index=False)
print('Datos guardados en ../data/sinprocesar/balance/balance-electrico.csv')

Extrayendo datos de balance del año 2019...
Extrayendo datos de balance del año 2020...
Extrayendo datos de balance del año 2021...
Extrayendo datos de balance del año 2022...
Extrayendo datos de balance del año 2023...
Extrayendo datos de balance del año 2024...
Datod de balance entre 2019 y 2024 extraidos:


,fecha,valor,tipo,energia,region
0,2019-01-01T00:00:00.000+01:00,53722.120,Hidráulica,Renovable,peninsular
1,2019-01-02T00:00:00.000+01:00,61294.768,Hidráulica,Renovable,peninsular
2,2019-01-03T00:00:00.000+01:00,82981.951,Hidráulica,Renovable,peninsular
3,2019-01-04T00:00:00.000+01:00,94301.940,Hidráulica,Renovable,peninsular
4,2019-01-05T00:00:00.000+01:00,58822.692,Hidráulica,Renovable,peninsular


Datos guardados en ../data/sinprocesar/balance/balance-electrico.csv


DEMANDA

In [42]:
### EVOLUCION ###

# Endpoint para la demanda
filtro_demanda = '/es/datos/demanda/evolucion'

endpoint = f"{URL}/{filtro_demanda}"
# Inicializamos un df vacío para almacenar los datos
df_demanda = pd.DataFrame()
# Iteramos sobre los años y regiones
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "day"

    print(f"Extrayendo datos de demanda del año {year}...")

    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        data = response.json()
        included = data.get('included', [])

        registros = []

        for serie in included:
            indicador = serie['attributes']['title']
            valores = serie['attributes'].get('values', [])
            # Extraer los valores de la serie
            for punto in valores:
                registros.append({
                    'fecha': punto['datetime'],
                    'valor': punto['value'],
                    'indicator': indicador,
                    'region': region_name,
                    'año': year
                })

        df_demanda_year = pd.DataFrame(registros)
        df_demanda = pd.concat([df_demanda, df_demanda_year], ignore_index=True)

print("Datos de demanda extraídos entre el 2019 y el 2024:")
display(df_demanda.head())
# Guardamos el DataFrame en un archivo CSV
df_demanda.to_csv('../data/sinprocesar/demanda/demanda-evolucion.csv', index=False)
print('Datos guardados en ../data/sinprocesar/demanda/demanda-evolucion.csv')

Extrayendo datos de demanda del año 2019...
Extrayendo datos de demanda del año 2020...
Extrayendo datos de demanda del año 2021...
Extrayendo datos de demanda del año 2022...
Extrayendo datos de demanda del año 2023...
Extrayendo datos de demanda del año 2024...
Datos de demanda extraídos entre el 2019 y el 2024:


,fecha,valor,indicator,region,año
0,2019-01-01T00:00:00.000+01:00,582949.806,Demanda,peninsular,2019
1,2019-01-02T00:00:00.000+01:00,742199.407,Demanda,peninsular,2019
2,2019-01-03T00:00:00.000+01:00,787764.963,Demanda,peninsular,2019
3,2019-01-04T00:00:00.000+01:00,796010.035,Demanda,peninsular,2019
4,2019-01-05T00:00:00.000+01:00,721029.651,Demanda,peninsular,2019


Datos guardados en ../data/sinprocesar/demanda/demanda-evolucion.csv


In [ ]:
### IRE GENERAL ###
filtro_demanda = '/es/datos/demanda/ire-general'
endpoint = f"{URL}/{filtro_demanda}"

# Inicializamos un DataFrame vacío
df_ire_general = pd.DataFrame()

# Iteramos sobre los años y regiones
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "month"  # importante: solo funciona con 'month'

    print(f"Extrayendo datos IRE General del año {year}...")

    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
            continue

        data = response.json()
        included = data.get('included', [])

        registros = []

        for serie in included:
            indicador = serie['attributes']['title']
            valores = serie['attributes'].get('values', [])
            for punto in valores:
                registros.append({
                    'fecha': punto['datetime'],
                    'valor': punto['value'],
                    'indicador': indicador,
                    'region': region_name,
                    'año': year
                })

        df_ire_year = pd.DataFrame(registros)
        df_ire_general = pd.concat([df_ire_general, df_ire_year], ignore_index=True)

print("Datos IRE General extraídos entre el 2019 y el 2024:")
display(df_ire_general.head())

# Guardamos en un solo archivo CSV
df_ire_general.to_csv('../data/sinprocesar/demanda/ire-general.csv', index=False)
print('Datos guardados en ../data/sinprocesar/demanda/ire-general.csv')

Extrayendo datos IRE General del año 2019...
Extrayendo datos IRE General del año 2020...
Extrayendo datos IRE General del año 2021...
Extrayendo datos IRE General del año 2022...
Extrayendo datos IRE General del año 2023...
Extrayendo datos IRE General del año 2024...
Datos IRE General extraídos entre el 2019 y el 2024:


,fecha,valor,indicador,region,año
0,2019-01-01T00:00:00.000+01:00,123.969,Índice general,peninsular,2019
1,2019-02-01T00:00:00.000+01:00,114.805,Índice general,peninsular,2019
2,2019-03-01T00:00:00.000+01:00,126.515,Índice general,peninsular,2019
3,2019-04-01T00:00:00.000+02:00,119.432,Índice general,peninsular,2019
4,2019-05-01T00:00:00.000+02:00,130.280,Índice general,peninsular,2019


Datos guardados en ../data/sinprocesar/demanda/ire-general.csv


In [ ]:
### IRE GENERAL ANUAL ###

filtro_demanda = '/es/datos/demanda/ire-general-anual'
endpoint = f"{URL}/{filtro_demanda}"

# Inicializamos un DataFrame vacío
df_ire_general = pd.DataFrame()

# Iteramos sobre los años y regiones
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "month"  # importante: solo funciona con 'month'

    print(f"Extrayendo datos IRE General del año {year}...")

    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
            continue

        data = response.json()
        included = data.get('included', [])

        registros = []

        for serie in included:
            indicador = serie['attributes']['title']
            valores = serie['attributes'].get('values', [])
            for punto in valores:
                registros.append({
                    'fecha': punto['datetime'],
                    'valor': punto['value'],
                    'indicador': indicador,
                    'region': region_name,
                    'año': year
                })

        df_ire_year = pd.DataFrame(registros)
        df_ire_general = pd.concat([df_ire_general, df_ire_year], ignore_index=True)

print("Datos IRE General anual extraídos entre el 2019 y el 2024:")
display(df_ire_general.head())

# Guardamos en un solo archivo CSV
df_ire_general.to_csv('../data/sinprocesar/demanda/ire-general-anual.csv', index=False)
print('Datos guardados en ../data/sinprocesar/demanda/ire-general-anual.csv')

Extrayendo datos IRE General del año 2019...
Extrayendo datos IRE General del año 2020...
Extrayendo datos IRE General del año 2021...
Extrayendo datos IRE General del año 2022...
Extrayendo datos IRE General del año 2023...
Extrayendo datos IRE General del año 2024...
Datos IRE General extraídos entre el 2019 y el 2024:


,fecha,valor,indicador,region,año
0,2019-01-01T00:00:00.000+01:00,123.96900,Índice general,peninsular,2019
1,2019-02-01T00:00:00.000+01:00,119.38700,Índice general,peninsular,2019
2,2019-03-01T00:00:00.000+01:00,121.76300,Índice general,peninsular,2019
3,2019-04-01T00:00:00.000+02:00,121.18025,Índice general,peninsular,2019
4,2019-05-01T00:00:00.000+02:00,123.00020,Índice general,peninsular,2019


Datos guardados en ../data/sinprocesar/demanda/ire-general-anual.csv


In [ ]:
### IRE SERVICIOS ANUAL ###

filtro_demanda = '/es/datos/demanda/ire-servicios-anual'
endpoint = f"{URL}/{filtro_demanda}"

# Inicializamos un DataFrame vacío
df_ire_general = pd.DataFrame()

# Iteramos sobre los años y regiones
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "month"  # importante: solo funciona con 'month'

    print(f"Extrayendo datos IRE General del año {year}...")

    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        if response.status_code != 200:
            print(f"Error {response.status_code} para región: {region_name} ({geo_id}), año {year}")
            continue

        data = response.json()
        included = data.get('included', [])

        registros = []

        for serie in included:
            indicador = serie['attributes']['title']
            valores = serie['attributes'].get('values', [])
            for punto in valores:
                registros.append({
                    'fecha': punto['datetime'],
                    'valor': punto['value'],
                    'indicador': indicador,
                    'region': region_name,
                    'año': year
                })

        df_ire_year = pd.DataFrame(registros)
        df_ire_general = pd.concat([df_ire_general, df_ire_year], ignore_index=True)

print("Datos IRE General extraídos entre el 2019 y el 2024:")
display(df_ire_general.head())

# Guardamos en un solo archivo CSV
df_ire_general.to_csv('../data/sinprocesar/demanda/ire-servicios-anual.csv', index=False)
print('Datos guardados en ../data/sinprocesar/demanda/ire-servicios-anual.csv')

Extrayendo datos IRE General del año 2019...
Extrayendo datos IRE General del año 2020...
Extrayendo datos IRE General del año 2021...
Extrayendo datos IRE General del año 2022...
Extrayendo datos IRE General del año 2023...
Extrayendo datos IRE General del año 2024...
Datos IRE General extraídos entre el 2019 y el 2024:


,fecha,valor,indicador,region,año
0,2019-01-01T00:00:00.000+01:00,123.361000,Índice servicios,peninsular,2019
1,2019-02-01T00:00:00.000+01:00,116.962500,Índice servicios,peninsular,2019
2,2019-03-01T00:00:00.000+01:00,117.548333,Índice servicios,peninsular,2019
3,2019-04-01T00:00:00.000+02:00,116.665750,Índice servicios,peninsular,2019
4,2019-05-01T00:00:00.000+02:00,118.109800,Índice servicios,peninsular,2019


Datos guardados en ../data/sinprocesar/demanda/ire-servicios-anual.csv


GENERACION

In [41]:
### ESTRUCTURA-GENERACION ###

# Endpoint para la generación
filtro_generacion = "/es/datos/generacion/estructura-generacion"
endpoint = f"{URL}{filtro_generacion}"

df_generacion = pd.DataFrame()
## Iterar sobre los años
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "day"

    print(f"Extrayendo datos de estructura-generacion del año {year}...")
    # Iterar sobre los geo_ids y nombres de regiones
    # para cada año y geo_id
    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        if response.status_code == 200:
            try:
                data = response.json()
                included = data.get('included', [])

                if included:
                    registros = []

                    # Iterar sobre los grupos de datos en 'included'
                    for grupo in included:
                        group_name = grupo['attributes'].get('title', 'Desconocido')
                        group_type = grupo['attributes'].get('type', 'Desconocido')
                        contenidos = grupo['attributes'].get('values', [])

                        # Iterar sobre los valores de generación
                        for punto in contenidos:
                            registros.append({
                                'datetime': punto.get('datetime', 'Desconocido'),
                                'value': punto.get('value', 0),
                                'percentage': punto.get('percentage', 0),
                                'indicator': group_name,
                                'region': region_name,
                                'Type': group_type,
                                'year': year
                            })

                    df_generacion_year = pd.DataFrame(registros)
                    df_generacion = pd.concat([df_generacion, df_generacion_year], ignore_index=True)

            except Exception as e:
                print("Contenido de la respuesta:", response.text)

        else:
            print(f"Error en la solicitud para {region_name} {year}. Código: {response.status_code}")

print('Datos de estructura-generacion del 2019 al 2024:')
display(df_generacion.head())
# Guardar en csv
df_generacion.to_csv('../data/sinprocesar/generacion/estructura-generacion.csv', index=False)
print('Datos guardados en ../data/sinprocesar/generacion/estructura-generacion.csv')

Extrayendo datos de estructura-generacion del año 2019...
Extrayendo datos de estructura-generacion del año 2020...
Extrayendo datos de estructura-generacion del año 2021...
Extrayendo datos de estructura-generacion del año 2022...
Extrayendo datos de estructura-generacion del año 2023...
Extrayendo datos de estructura-generacion del año 2024...
Datos de estructura-generacion del 2019 al 2024:


,datetime,value,percentage,indicator,region,Type,year
0,2019-01-01T00:00:00.000+01:00,53722.120,0.097735,Hidráulica,peninsular,Renovable,2019
1,2019-01-02T00:00:00.000+01:00,61294.768,0.086331,Hidráulica,peninsular,Renovable,2019
2,2019-01-03T00:00:00.000+01:00,82981.951,0.115989,Hidráulica,peninsular,Renovable,2019
3,2019-01-04T00:00:00.000+01:00,94301.940,0.125121,Hidráulica,peninsular,Renovable,2019
4,2019-01-05T00:00:00.000+01:00,58822.692,0.086633,Hidráulica,peninsular,Renovable,2019


Datos guardados en ../data/sinprocesar/generacion/estructura-generacion.csv


In [40]:
### Renovable/No Renovable ###

# Endpoint para la generación
filtro_generacion = "/es/datos/generacion/evolucion-renovable-no-renovable"
endpoint = f"{URL}{filtro_generacion}"

df_generacion = pd.DataFrame()
## Iterar sobre los años
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "day"

    print(f"Extrayendo datos de evolucion renovable/No renovable del año {year}...")
    # Iterar sobre los geo_ids y nombres de regiones
    # para cada año y geo_id
    for geo_id, region_name in region_ids.items():
        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
            "geo_id": geo_id
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        if response.status_code == 200:
            try:
                data = response.json()
                included = data.get('included', [])

                if included:
                    registros = []

                    # Iterar sobre los grupos de datos en 'included'
                    for grupo in included:
                        group_name = grupo['attributes'].get('title', 'Desconocido')
                        group_type = grupo['attributes'].get('type', 'Desconocido')
                        contenidos = grupo['attributes'].get('values', [])

                        # Iterar sobre los valores de generación
                        for punto in contenidos:
                            registros.append({
                                'datetime': punto.get('datetime', 'Desconocido'),
                                'value': punto.get('value', 0),
                                'percentage': punto.get('percentage', 0),
                                'indicator': group_name,
                                'region': region_name,
                                'Type': group_type,
                                'year': year
                            })

                    df_generacion_year = pd.DataFrame(registros)
                    df_generacion = pd.concat([df_generacion, df_generacion_year], ignore_index=True)

            except Exception as e:
                print("Contenido de la respuesta:", response.text)

        else:
            print(f"Error en la solicitud para {region_name} {year}. Código: {response.status_code}")

print('Datos de generación del 2019 al 2024:')
display(df_generacion.head())
# Guardar en csv
df_generacion.to_csv('../data/sinprocesar/generacion/evolucion-renovable-NoRenovable.csv', index=False)
print('Datos guardados en ../data/sinprocesar/generacion/evolucion-renovable-NoRenovable.csv')

Extrayendo datos de evolucion renovable/No renovable del año 2019...
Extrayendo datos de evolucion renovable/No renovable del año 2020...
Extrayendo datos de evolucion renovable/No renovable del año 2021...
Extrayendo datos de evolucion renovable/No renovable del año 2022...
Extrayendo datos de evolucion renovable/No renovable del año 2023...
Extrayendo datos de evolucion renovable/No renovable del año 2024...
Datos de generación del 2019 al 2024:


,datetime,value,percentage,indicator,region,Type,year
0,2019-01-01T00:00:00.000+01:00,169351.7695,0.308096,Renovable,peninsular,Renovable,2019
1,2019-01-02T00:00:00.000+01:00,294297.6145,0.414504,Renovable,peninsular,Renovable,2019
2,2019-01-03T00:00:00.000+01:00,229057.3825,0.320167,Renovable,peninsular,Renovable,2019
3,2019-01-04T00:00:00.000+01:00,210376.4825,0.279131,Renovable,peninsular,Renovable,2019
4,2019-01-05T00:00:00.000+01:00,214172.6470,0.315429,Renovable,peninsular,Renovable,2019


Datos guardados en ../data/sinprocesar/generacion/evolucion-renovable-NoRenovable.csv


INTERCAMBIO

In [39]:
### FRONTERAS ###

# Lista de países
lista_paises = ['francia-frontera', 'portugal-frontera', 'marruecos-frontera', 'andorra-frontera']

datos_intercambios = []

for pais in lista_paises:
    print(f'Consultando datos de {pais}...')
    # Endpoint para intercambios
    filtro_intercambio = f"/es/datos/intercambios/{pais}"
    endpoint = f"{URL}{filtro_intercambio}"
    # Iterar sobre los años y extraer datos de cada año
    # y cada país para el año 2019 al 2024
    for year in range(2019, 2025):
        start_date = f"{year}-01-01T00:00"
        end_date = f"{year}-12-31T23:59"
        TIME_TRUNC = "day"

        print(f"Extrayendo datos del año {year}...")

        params = {
            "start_date": start_date,
            "end_date": end_date,
            "time_trunc": TIME_TRUNC,
        }

        response = requests.get(endpoint, headers=HEADERS, params=params)

        if response.status_code == 200:
            data = response.json()
            # Verificar si hay datos disponibles
            valores = data["included"][0]["attributes"].get("values", [])
            if valores:
                df = pd.DataFrame(valores)
                df["pais"] = pais
                df["año"] = year
                datos_intercambios.append(df)
        else:
            print(f"Error en la solicitud para {pais} en {year}. Código: {response.status_code}")

# Unir todos los datos en un único DataFrame
df_intercambio = pd.concat(datos_intercambios, ignore_index=True)

print(f"Datos de intercambios entre paises del 2019 al 2024:")
display(df_intercambio.head())
# Guardar datos en CSV
df_intercambio.to_csv('../data/sinprocesar/intercambio/fronteras.csv', index=False)
print('Datos guardados en ../data/sinprocesar/intercambio/fronteras.csv')

Consultando datos de francia-frontera...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Consultando datos de portugal-frontera...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Consultando datos de marruecos-frontera...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Consultando datos de andorra-frontera...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Datos de intercambios entre paises del 20

,value,percentage,datetime,pais,año
0,-208.239,0.003465,2019-01-01T00:00:00.000+01:00,francia-frontera,2019
1,-2325.553,0.036857,2019-01-02T00:00:00.000+01:00,francia-frontera,2019
2,-11123.719,0.243767,2019-01-03T00:00:00.000+01:00,francia-frontera,2019
3,-17758.093,0.397978,2019-01-04T00:00:00.000+01:00,francia-frontera,2019
4,-4267.317,0.102386,2019-01-05T00:00:00.000+01:00,francia-frontera,2019


Datos guardados en ../data/sinprocesar/intercambio/fronteras.csv


In [38]:
### ENLACE BALEARES ###

datos_intercambios = []


print(f'Consultando datos de baleares...')
# Endpoint para intercambios
filtro_intercambio = f"/es/datos/intercambios/enlace-baleares"
endpoint = f"{URL}{filtro_intercambio}"
# Iterar sobre los años y extraer datos de cada año
# del enlace con baleares para el año 2019 al 2024
for year in range(2019, 2025):
    start_date = f"{year}-01-01T00:00"
    end_date = f"{year}-12-31T23:59"
    TIME_TRUNC = "day"

    print(f"Extrayendo datos del año {year}...")

    params = {
        "start_date": start_date,
        "end_date": end_date,
        "time_trunc": TIME_TRUNC,
    }

    response = requests.get(endpoint, headers=HEADERS, params=params)

    if response.status_code == 200:
        data = response.json()
        # Verificar si hay datos disponibles
        valores = data["included"][0]["attributes"].get("values", [])
        if valores:
            df = pd.DataFrame(valores)
            df["pais"] = pais
            df["año"] = year
            datos_intercambios.append(df)
    else:
        print(f"Error en la solicitud para {pais} en {year}. Código: {response.status_code}")

# Unir todos los datos en un único DataFrame
df_intercambio = pd.concat(datos_intercambios, ignore_index=True)

print(f"Datos de intercambios del enlace baleares del 2019 al 2024:")
display(df_intercambio.head())
# Guardar datos en CSV
df_intercambio.to_csv('../data/sinprocesar/intercambio/enlace-baleares.csv', index=False)
print('Datos guardados en ../data/sinprocesar/intercambio/enlace-baleares.csv')

Consultando datos de baleares...
Extrayendo datos del año 2019...
Extrayendo datos del año 2020...
Extrayendo datos del año 2021...
Extrayendo datos del año 2022...
Extrayendo datos del año 2023...
Extrayendo datos del año 2024...
Datos de intercambios del enlace baleares del 2019 al 2024:


,value,percentage,datetime,pais,año
0,-3181.766,0.5,2019-01-01T00:00:00.000+01:00,andorra-frontera,2019
1,-3959.886,0.5,2019-01-02T00:00:00.000+01:00,andorra-frontera,2019
2,-4407.091,0.5,2019-01-03T00:00:00.000+01:00,andorra-frontera,2019
3,-4373.999,0.5,2019-01-04T00:00:00.000+01:00,andorra-frontera,2019
4,-3870.764,0.5,2019-01-05T00:00:00.000+01:00,andorra-frontera,2019


Datos guardados en ../data/sinprocesar/intercambio/enlace-baleares.csv


In [263]:
################################################################################################################################